In [1]:
BRANCH = 'r1.6.0'

In [2]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
BRANCH = 'r1.6.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.6.0) to /tmp/pip-install-roxlq8_r/nemo-toolkit_781d738bf03f42f1b20b94a8e7fba8d7
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-roxlq8_r/nemo-toolkit_781d738bf03f42f1b20b94a8e7fba8d7
  Running command git checkout -b r1.6.0 --track origin/r1.6.0
  Switched to a new branch 'r1.6.0'
  Branch 'r1.6.0' set up to track remote branch 'r1.6.0' from 'origin'.
  Resolved https://github.com/NVIDIA/NeMo.git to commit 4b46445d08611686e0270e18870244df3f53ce2e


In [3]:
import os
import wget
from nemo.collections import nlp as nemo_nlp
from nemo.collections import common as nemo_common
from omegaconf import OmegaConf

[NeMo W 2022-01-20 22:08:56 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2022-01-20 22:08:57 experimental:27] Module <function get_argmin_mat at 0x7f74c6081430> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-20 22:08:57 experimental:27] Module <function getMultiScaleCosAffinityMatrix at 0x7f74c60814c0> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-20 22:08:57 experimental:27] Module <function parse_scale_configs at 0x7f74c6084040> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-01-20 22:08:57 experimental:27] Module <function get_embs_and_timestamps at 0x7f74c60815e0> is experim

# Tokenizers Background

For Natural Language Processing, tokenization is an essential part of data preprocessing. It is the process of splitting a string into a list of tokens. One can think of token as parts like a word is a token in a sentence.
Depending on the application, different tokenizers are more suitable than others. 


For example, a WordTokenizer that splits the string on any whitespace, would tokenize the following string 

"My first program, Hello World." -> ["My", "first", "program,", "Hello", "World."]

To turn the tokens into numerical model input, the standard method is to use a vocabulary and one-hot vectors for [word embeddings](https://en.wikipedia.org/wiki/Word_embedding). If a token appears in the vocabulary, its index is returned, if not the index of the unknown token is returned to mitigate out-of-vocabulary (OOV).




# Tokenizers in NeMo

In NeMo, we support the most used tokenization algorithms. We offer a wrapper around [Hugging Faces's AutoTokenizer](https://huggingface.co/transformers/model_doc/auto.html#autotokenizer) - a factory class that gives access to all Hugging Face tokenizers. This includes particularly all BERT-like model tokenizers, such as BertTokenizer, AlbertTokenizer, RobertaTokenizer, GPT2Tokenizer. Apart from that, we also support other tokenizers such as WordTokenizer, CharTokenizer, and [Google's SentencePieceTokenizer](https://github.com/google/sentencepiece).  


We make sure that all tokenizers are compatible with BERT-like models, e.g. BERT, Roberta, Albert, and Megatron. For that, we provide a high-level user API `get_tokenizer()`, which allows the user to instantiate a tokenizer model with only four input arguments: 
* `tokenizer_name: str`
* `tokenizer_model: Optional[str] = None`
* `vocab_file: Optional[str] = None`
* `special_tokens: Optional[Dict[str, str]] = None`

Hugging Face and Megatron tokenizers (which uses Hugging Face underneath) can be automatically instantiated by only `tokenizer_name`, which downloads the corresponding `vocab_file` from the internet. 

For SentencePieceTokenizer, WordTokenizer, and CharTokenizers `tokenizer_model` or/and `vocab_file` can be generated offline in advance using [`scripts/tokenizers/process_asr_text_tokenizer.py`](https://github.com/NVIDIA/NeMo/blob/stable/scripts/process_asr_text_tokenizer.py)

The tokenizers in NeMo are designed to be used interchangeably, especially when
used in combination with a BERT-based model.

Let's take a look at the list of available tokenizers:

In [4]:
nemo_nlp.modules.get_tokenizer_list()

[NeMo W 2022-01-20 22:08:57 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/transformers/models/auto/configuration_auto.py:368: FutureWarning: ALL_PRETRAINED_CONFIG_ARCHIVE_MAP is deprecated and will be removed in v5 of Transformers. It does not contain all available model checkpoints, far from it. Checkout hf.co/models for that.
      warnings.warn(
    


['sentencepiece',
 'char',
 'word',
 'bert-large-cased',
 'cl-tohoku/bert-base-japanese-char',
 'google/bigbird-pegasus-large-bigpatent',
 'google/electra-small-generator',
 'google/canine-s',
 'distilbert-base-multilingual-cased',
 'roberta-base',
 'xlm-mlm-enro-1024',
 'google/electra-base-discriminator',
 'funnel-transformer/medium',
 'TurkuNLP/bert-base-finnish-uncased-v1',
 'asapp/sew-d-tiny-100k',
 'allenai/led-base-16384',
 'xlm-mlm-tlm-xnli15-1024',
 'distilroberta-base',
 'allenai/longformer-large-4096',
 'xlm-roberta-base',
 'openai/imagegpt-medium',
 'openai-gpt',
 'tau/splinter-base',
 'bert-base-multilingual-uncased',
 'facebook/deit-base-distilled-patch16-224',
 'EleutherAI/gpt-neo-1.3B',
 'ctrl',
 'facebook/bart-large',
 't5-small',
 'funnel-transformer/xlarge-base',
 'google/fnet-base',
 'rembert',
 'facebook/unispeech_sat-base-960h',
 'google/tapas-base-finetuned-wikisql-supervised',
 'distilbert-base-cased-distilled-squad',
 'umberto-commoncrawl-cased-v1',
 'facebook/

# Hugging Face AutoTokenizer

In [5]:
# instantiate tokenizer wrapper using pretrained model name only
tokenizer1 = nemo_nlp.modules.get_tokenizer(tokenizer_name="bert-base-cased")

# the wrapper has a reference to the original HuggingFace tokenizer
print(tokenizer1.tokenizer)

[NeMo I 2022-01-20 22:08:57 tokenizer_utils:123] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-cased, vocab_file: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


PreTrainedTokenizer(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [6]:
# check vocabulary (this can be very long)
print(tokenizer1.tokenizer.vocab)

OrderedDict([('[PAD]', 0), ('[unused1]', 1), ('[unused2]', 2), ('[unused3]', 3), ('[unused4]', 4), ('[unused5]', 5), ('[unused6]', 6), ('[unused7]', 7), ('[unused8]', 8), ('[unused9]', 9), ('[unused10]', 10), ('[unused11]', 11), ('[unused12]', 12), ('[unused13]', 13), ('[unused14]', 14), ('[unused15]', 15), ('[unused16]', 16), ('[unused17]', 17), ('[unused18]', 18), ('[unused19]', 19), ('[unused20]', 20), ('[unused21]', 21), ('[unused22]', 22), ('[unused23]', 23), ('[unused24]', 24), ('[unused25]', 25), ('[unused26]', 26), ('[unused27]', 27), ('[unused28]', 28), ('[unused29]', 29), ('[unused30]', 30), ('[unused31]', 31), ('[unused32]', 32), ('[unused33]', 33), ('[unused34]', 34), ('[unused35]', 35), ('[unused36]', 36), ('[unused37]', 37), ('[unused38]', 38), ('[unused39]', 39), ('[unused40]', 40), ('[unused41]', 41), ('[unused42]', 42), ('[unused43]', 43), ('[unused44]', 44), ('[unused45]', 45), ('[unused46]', 46), ('[unused47]', 47), ('[unused48]', 48), ('[unused49]', 49), ('[unused50

In [7]:
# show all special tokens if it has any
print(tokenizer1.tokenizer.all_special_tokens)

['[CLS]', '[SEP]', '[UNK]', '[PAD]', '[MASK]']


In [8]:
# instantiate tokenizer using custom vocabulary
vocab_file = "myvocab.txt"
vocab = ["he", "llo", "world"]
with open(vocab_file, 'w') as vocab_fp:
  vocab_fp.write("\n".join(vocab))

In [9]:
tokenizer2 = nemo_nlp.modules.get_tokenizer(tokenizer_name="bert-base-cased", vocab_file=vocab_file)

[NeMo I 2022-01-20 22:09:02 tokenizer_utils:123] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-cased, vocab_file: myvocab.txt, special_tokens_dict: {}, and use_fast: False


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


In [10]:
# Since we did not overwrite special tokens they should be the same as before
print(tokenizer1.tokenizer.all_special_tokens == tokenizer2.tokenizer.all_special_tokens )

True


## Adding Special tokens

We do not recommend overwriting special tokens for Hugging Face pretrained models, 
since these are the commonly used default values. 

If a user still wants to overwrite the special tokens, specify some of the following keys:

In [11]:
special_tokens_dict = {"unk_token": "<UNK>", 
                       "sep_token": "<SEP>", 
                       "pad_token": "<PAD>", 
                       "bos_token": "<CLS>", 
                       "mask_token": "<MASK>",
                       "eos_token": "<SEP>",
                       "cls_token": "<CLS>"}
tokenizer3 = nemo_nlp.modules.get_tokenizer(tokenizer_name="bert-base-cased",
                                            vocab_file=vocab_file,
                                            special_tokens=special_tokens_dict)

# print newly set special tokens
print(tokenizer3.tokenizer.all_special_tokens)
# the special tokens should be different from the previous special tokens
print(tokenizer3.tokenizer.all_special_tokens != tokenizer1.tokenizer.all_special_tokens )

[NeMo I 2022-01-20 22:09:06 tokenizer_utils:123] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-cased, vocab_file: myvocab.txt, special_tokens_dict: {'unk_token': '<UNK>', 'sep_token': '<SEP>', 'pad_token': '<PAD>', 'bos_token': '<CLS>', 'mask_token': '<MASK>', 'eos_token': '<SEP>', 'cls_token': '<CLS>'}, and use_fast: False


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[NeMo W 2022-01-20 22:09:10 auto_tokenizer:145] ['<MASK>', '<CLS>', '<SEP>', '<PAD>', '<SEP>', '<CLS>', '<UNK>'] 
     will be added to the vocabulary.
    Please resize your model accordingly, see NLP_Tokenizers.ipynb for more details.


[NeMo I 2022-01-20 22:09:10 auto_tokenizer:171] 4 special tokens added, resize your model accordingly.
['<CLS>', '<SEP>', '<UNK>', '<PAD>', '<MASK>']
True


Notice, that if you specify tokens that were not previously included in the tokenizer's vocabulary file, new tokens will be added to the vocabulary file. You will see a message like this:

`['<MASK>', '<CLS>', '<SEP>', '<PAD>', '<SEP>', '<CLS>', '<UNK>'] 
     will be added to the vocabulary.
    Please resize your model accordingly`

In [ ]:
# A safer way to add special tokens is the following:

# define your model
pretrained_model_name = 'bert-base-uncased'
model = nemo_nlp.modules.get_lm_model(pretrained_model_name=pretrained_model_name)

# define pretrained tokenizer
tokenizer_default = nemo_nlp.modules.get_tokenizer(tokenizer_name=pretrained_model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer_default.text_to_tokens('<MY_NEW_TOKEN> and another word')

As you can see in the above, the tokenizer splits `<MY_NEW_TOKEN>` into subtokens. Let's add this to the special tokens to make sure the tokenizer does not split this into subtokens.

In [ ]:
special_tokens = {'bos_token': '<BOS>',
                  'cls_token': '<CSL>',
                  'additional_special_tokens': ['<MY_NEW_TOKEN>', '<ANOTHER_TOKEN>']}
tokenizer_default.add_special_tokens(special_tokens_dict=special_tokens)

# resize your model so that the embeddings for newly added tokens are updated during training/finetuning
model.resize_token_embeddings(tokenizer_default.vocab_size)

# let's make sure the tokenizer doesn't split our special tokens into subtokens
tokenizer_default.text_to_tokens('<MY_NEW_TOKEN> and another word')

Now, the model doesn't break down our special token into the subtokens.

## Megatron model tokenizer

In [ ]:
# Megatron tokenizers are instances of the Hugging Face BertTokenizer. 
tokenizer4 = nemo_nlp.modules.get_tokenizer(tokenizer_name="megatron-bert-cased")

# Train custom tokenizer model and vocabulary from text file 

We use the [`scripts/tokenizers/process_asr_text_tokenizer.py`](https://github.com/NVIDIA/NeMo/blob/stable/scripts/process_asr_text_tokenizer.py) script to create a custom tokenizer model with its own vocabulary from an input file

In [ ]:
# download tokenizer script
script_file = "process_asr_text_tokenizer.py"

if not os.path.exists(script_file):
    print('Downloading script file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/tokenizers/process_asr_text_tokenizer.py')
else:
    print ('Script already exists')

In [ ]:
# Let's prepare some small text data for the tokenizer
data_text = "NeMo is a toolkit for creating Conversational AI applications. \
NeMo toolkit makes it possible for researchers to easily compose complex neural network architectures \
for conversational AI using reusable components - Neural Modules. \
Neural Modules are conceptual blocks of neural networks that take typed inputs and produce typed outputs. \
Such modules typically represent data layers, encoders, decoders, language models, loss functions, or methods of combining activations. \
The toolkit comes with extendable collections of pre-built modules and ready-to-use models for automatic speech recognition (ASR), \
natural language processing (NLP) and text synthesis (TTS). \
Built for speed, NeMo can utilize NVIDIA's Tensor Cores and scale out training to multiple GPUs and multiple nodes."

In [ ]:
# Write the text data into a file
data_file="data.txt"

with open(data_file, 'w') as data_fp:
  data_fp.write(data_text)

In [ ]:
# Some additional parameters for the tokenizer
# To tokenize at unigram, char or word boundary instead of using bpe, change --spe_type accordingly. 
# More details see https://github.com/google/sentencepiece#train-sentencepiece-model

tokenizer_spe_type = "bpe"  # <-- Can be `bpe`, `unigram`, `word` or `char`
vocab_size = 32

In [ ]:
! python process_asr_text_tokenizer.py --data_file=$data_file --data_root=. --vocab_size=$vocab_size --tokenizer=spe --spe_type=$tokenizer_spe_type

In [ ]:
# See created tokenizer model and vocabulary
spe_model_dir=f"tokenizer_spe_{tokenizer_spe_type}_v{vocab_size}"
! ls $spe_model_dir

# Use custom tokenizer for data preprocessing
## Example: SentencePiece for BPE

In [ ]:
# initialize tokenizer with created tokenizer model, which inherently includes the vocabulary and specify optional special tokens
tokenizer_spe = nemo_nlp.modules.get_tokenizer(tokenizer_name="sentencepiece", tokenizer_model=spe_model_dir+"/tokenizer.model", special_tokens=special_tokens_dict)

# specified special tokens are added to the vocabuary
print(tokenizer_spe.vocab_size)

# Using any tokenizer to tokenize text into BERT compatible input


In [ ]:
text="hello world"

# create tokens
tokenized = [tokenizer_spe.bos_token] + tokenizer_spe.text_to_tokens(text) + [tokenizer_spe.eos_token]
print(tokenized)

# turn token into input_ids for a neural model, such as BERTModule

print(tokenizer_spe.tokens_to_ids(tokenized))